In [ ]:
import pandas as pd
import numpy as np
from io import StringIO

<H4>
Files imported: /data/PowerSchoolSFTP/Students_export.csv, /data/PowerSchoolSFTP/ContactsPowerQuery.csv <br>
File exported: StudentContactInfo.csv <br>
<br>
Product is the necessary Student & Parent/Guardian contact info the eIEPPROStudentDemographics.csv requires

The plan: <br>
 * get the contact fields (using custody flag) per student <br>
 * put in order of student id (new index) <br> 
 * pick first mother, father, guardian contacts <br> 
 * flush out required fields and transpose <br>
 * join them with the "Active Child Find" students from the other file <br>
<br>
Key = Student Number, STUDENTS.Student_Number from Students_exp <br>

Spec for - concentrating on Parents in this file 

| Data Value | e-IEPPRO Column Name | Details | 
| --- |--- |--- |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |

| --- |--- |--- |
| --- |--- |--- |

In [ ]:
# StudentAddress  = pd.read_csv("Students_export.csv", encoding='latin-1') 
StudentContacts = pd.read_csv("ContactsPowerQuery.csv", encoding='latin-1')

In [ ]:
# Contacts for _ALL_ students in PS
StudentContacts

In [ ]:
StudentContacts.info()

In [ ]:
def BuildParentContacts( df_ContactsPQ ):
    global df_PContacts
    df_PContacts = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_ContactsPQ.iterrows():
        parent_id = row['Contact ID']
        student_ID = row['Student Number']
        relationship = row['Relationship'] 
        ParentName = row['Contact First Name'] + ' ' + row['Contact Last Name']
        PhoneNum = row['Phone Number']
        
        df_PContacts.at[index, 'student_ID'] = student_ID
        
        match relationship:
            case 'Mother':
                df_PContacts.at[index, 'Mother'] = parent_id
                df_PContacts.at[index, 'Parent1Name'] = ParentName
            case 'Father':
                df_PContacts.at[index, 'Father'] = parent_id
                df_PContacts.at[index, 'Parent2Name'] = ParentName
            case 'Guardian':
                df_PContacts.at[index, 'Guardian'] = parent_id 
                df_PContacts.at[index, 'GuardianName'] = ParentName
    return df_PContacts

In [ ]:
# Fields e-IEPPRO wants are Parent# Name, Address, Home Phone, Work Phone, Cell Phone, Relationship, Email
# Fields the ContactsQuery offers are: Name, "Phone Number", Relationship, Email Address

def BuildParentContactsV2( df_ContactsPQ ):
    global df_PContacts
    df_PContacts = pd.DataFrame()
    """ Create a counter for each student ID """
    for index, row in df_ContactsPQ.iterrows():
        parent_id = row['Contact ID']
        student_ID = row['Student Number']
        relationship = row['Relationship'] 
        ParentName = row['Contact First Name'] + ' ' + row['Contact Last Name']
        ParentPhone = row['Phone Number']
        ParentEmail = row['Email Address']
        
        df_PContacts.at[index, 'student_ID'] = student_ID
        
        match relationship:
            case 'Mother':
                df_PContacts.at[index, 'Mother'] = parent_id
                df_PContacts.at[index, 'Parent1Name'] = ParentName
                df_PContacts.at[index, 'Parent1CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent1Email'] = ParentEmail
            case 'Father':
                df_PContacts.at[index, 'Father'] = parent_id
                df_PContacts.at[index, 'Parent2Name'] = ParentName
                df_PContacts.at[index, 'Parent2CellPhone'] = ParentPhone
                df_PContacts.at[index, 'Parent2Email'] = ParentEmail
            case 'Guardian':
                df_PContacts.at[index, 'Guardian'] = parent_id  
                df_PContacts.at[index, 'GuardianName'] = ParentName
                df_PContacts.at[index, 'GuardianCellPhone'] = ParentPhone
                df_PContacts.at[index, 'GuardianEmail'] = ParentEmail
    return df_PContacts

In [ ]:
# Runs in ~ 9 minutes 27 seconds minutes on my local computer
# Takes all Contacts and creates ~100,364 rows
# Efficiency measure: pull 504Idea community first?
TestContacts = BuildParentContactsV2(StudentContacts)

In [ ]:
TestContacts

In [ ]:
TestContacts.to_csv('TestContacts.csv', encoding='utf-8')

In [ ]:
df_StudentContactsToWrite = TestContacts.groupby('student_ID').agg('first')

In [ ]:
#  Down to 21,522 rows now (approx our AZ student body, we needed this data, regardless)
df_StudentContactsToWrite

In [ ]:
df_StudentContactsToWrite.to_csv('StudentContactsBeta.csv', encoding='utf-8')

<H2> The join below results in 4,207 students and parent contacts matched

In [ ]:
df_504_ID_Contacts = df_StudentInfo_504_idea.join(df_CustConStuNum, on='Student Number').drop_duplicates()

In [ ]:
df_504_ID_Contacts

In [ ]:
df_504_ID_Contacts.info()

In [ ]:
df_504_ID_Contacts

In [ ]:
# Results in 4,207 Contacts - some students have one, some three, mostly two
df_504_ID_Contacts.to_csv('eIEPPRO_Contacts.csv', encoding='utf-8')

<H1> Break here

In [ ]:
# Next up, answer questions:
# Who is not in 504_idea that is in this file?
# who is in 504_idea that is not in this file?
# Let's start with reorganizing the file for student name visibility

In [ ]:
df_StudentInfo_504_idea.info()

In [ ]:
list_504_and_idea = df_StudentInfo_504_idea['Student Number'].reset_index(drop=True)

In [ ]:
list_504_and_idea

In [ ]:
df_CustodyContacts['ParentName'] =  df_CustodyContacts['Contact Last Name'].map(str) + ", " + df_CustodyContacts['Contact First Name'].map(str)

Need to determine "Two Best" contacts

| Data Value | e-IEPPRO Column Name | Details | 
| --- |--- |--- |
| Student Address #1          | StudentAddress1     | Powerschool STUDENTS.Street ~ Street Address |
| Student Address #2          | StudentAddress2     | City, State and Zip Code |
| Student Physical Address    | StudentPhyAddress   | --- |  
| Student Phone #             | StudentPhone        | --- |
| Student E-Mail              | StudentEmail        | --- |
| Parent #1 Name              | Parent1Name         | --- |
| Parent #1 Address-1         | Parent1Address1     | Street Address |
| Parent #1 Address-2         | Parent1Address2     | City, State and Zip Code |  
| Parent #1 Home Phone        | Parent1HomePhone    | --- |
| Parent #1 Work Phone        | Parent1WorkPhone    | --- |
| Parent #1 Cell Phone        | Parent1CellPhone    | --- |
| Parent #1 Relationship      | Parent1Relationship | --- |
| Parent #1 Email             | Parent1Email        | --- |
| Parent #2 Name              | Parent2Name         | --- |
| Parent #2 Address-1         | Parent2Address1     | Street Address |
| Parent #2 Address-2         | Parent2Address2     | City, State and Zip Code |  
| Parent #2 Home Phone        | Parent2HomePhone    | --- |
| Parent #2 Work Phone        | Parent2WorkPhone    | --- |
| Parent #2 Cell Phone        | Parent2CellPhone    | --- |
| Parent #2 Relationship      | Parent2Relationship | --- |
| Parent #2 Email             | Parent2Email        | --- |

| --- |--- |--- |

In [ ]:
df_ContactsPivot = df_CustodyContacts.copy()

In [ ]:
df_ContactsPivot.info()

In [ ]:
df_ContactsbyStudentNum = df_ContactsPivot[['Student Number', 'Student LastFirst', 'Contact ID', 'ParentName', 'Relationship', 
                                           'Email Address', 'Phone Number']].drop_duplicates().reset_index(drop=True)

In [ ]:
df_ContactsbyStudentNum

In [ ]:
df_ContactsbyStudentNum.to_csv('df_ContactsbyStudentNum.csv', encoding='utf-8')

In [ ]:
df_ContactsbyStudentNum['Student Number'].value_counts()

In [ ]:
df_ContactsPivot.pivot(index='Student Number', columns= ['Contact First Name', 'Contact Last Name'], values = ['Contact First Name', 'Contact Last Name'])